In [28]:
import pandas as pd

PREGUNTAS = pd.read_excel('../resultados_azure/CATALOGO/PREGUNTAS.xls')
CANTON = pd.read_excel('../resultados_azure/CATALOGO/CANTON.xls')
CIRCUNSCRIPCION = pd.read_excel('../resultados_azure/CATALOGO/CIRCUNSCRIPCION.xls')
DIGNIDAD = pd.read_excel('../resultados_azure/CATALOGO/DIGNIDAD.xls')
JUNTA = pd.read_excel('../resultados_azure/CATALOGO/JUNTA.xls')
PARROQUIA = pd.read_excel('../resultados_azure/CATALOGO/PARROQUIA.xls')
OPCIONES = pd.read_excel('../resultados_azure/CATALOGO/OPCIONES.xls')
PROVINCIA = pd.read_excel('../resultados_azure/CATALOGO/PROVINCIA.xls')
ZONA = pd.read_excel('../resultados_azure/CATALOGO/ZONA.xls')

bases_azure = [
    "../resultados_azure/A_REFERENDUM_PREGUNTA_2024.04.25_01.50.01.txt",
    "../resultados_azure/B_REFERENDUM_PREGUNTA1_2024.04.25_01.50.02.txt",
    "../resultados_azure/C_REFERENDUM_PREGUNTA2_2024.04.25_01.50.03.txt",
    "../resultados_azure/D_REFERENDUM_PREGUNTA3_2024.04.25_01.50.04.txt",
    "../resultados_azure/E_REFERENDUM_PREGUNTA4_2024.04.25_01.50.05.txt",
    "../resultados_azure/F_CONSULTAPOPULAR_PREGUNTA1_2024.04.25_01.55.06.txt",
    "../resultados_azure/G_CONSULTAPOPULAR_PREGUNTA2_2024.04.25_01.55.08.txt",
    "../resultados_azure/H_CONSULTAPOPULAR_PREGUNTA3_2024.04.25_01.55.09.txt",
    "../resultados_azure/I_CONSULTAPOPULAR_PREGUNTA4_2024.04.25_01.55.09.txt",
    '../resultados_azure/J_CONSULTAPOPULAR_PREGUNTA5_2024.04.25_01.55.10.txt',
    '../resultados_azure/K_CONSULTAPOPULAR_PREGUNTA6_2024.04.25_01.55.11.txt'
]

final = pd.DataFrame()
for i in bases_azure:
    lectura =  pd.read_table(i,sep='|',names= 
                             ['COD_DIGNIDAD','COD_PROVINCIA',
                              'COD_CIRCUNSCRIPCION','COD_CANTON','COD_PARROQUIA',
                              'COD_ZONA','NUM_JUNTA','SEX_JUNTA','COD_JUNTA',
                              'COD_PREGUNTA','COD_OPCION','NUM_SUF_ACTA','BLANCOS','NULOS','FIN_RESULTADO'])
    final = pd.concat([final,lectura])


baseP = final

baseP = pd.merge(DIGNIDAD,baseP,on='COD_DIGNIDAD')
baseP = pd.merge(PROVINCIA,baseP,on='COD_PROVINCIA')
baseP = pd.merge(CANTON[['COD_CANTON','NOM_CANTON']],baseP,on='COD_CANTON')
baseP = pd.merge(PARROQUIA[['COD_PARROQUIA','NOM_PARROQUIA']],baseP,on='COD_PARROQUIA')
#baseP = pd.merge(Est_parroquia,baseP,on='COD_PARROQUIA')
baseP = pd.merge(PREGUNTAS[['COD_PREGUNTA','NOM_PREGUNTA','LIS_PREGUNTA']],baseP,on='COD_PREGUNTA')
baseP = pd.merge(OPCIONES[['COD_OPCION','NOM_OPCION']],baseP,on='COD_OPCION')
resultadosP = baseP.groupby(by = ['NOM_PREGUNTA','NOM_PROVINCIA','NOM_OPCION']).sum(numeric_only=True)[['FIN_RESULTADO']].reset_index()

preguntas = resultadosP['NOM_PREGUNTA'].unique()
provincias = resultadosP['NOM_PROVINCIA'].unique()



resultados_porcentajes = pd.DataFrame()
for pregunta in preguntas:
    for provincia in provincias:
        aux = resultadosP[resultadosP['NOM_PREGUNTA']==pregunta]
        aux = aux[aux['NOM_PROVINCIA']==provincia]
        total = aux['FIN_RESULTADO'].sum()
        aux['PORCENTAJE'] = aux['FIN_RESULTADO']/total
        aux['%'] = aux['PORCENTAJE'].apply(lambda x: f"{x:.2%}")
        
def proporcion(NO,SI):
    if SI == 0:
        return NO,'NO'
    elif NO>=SI:
        return NO/SI, 'NO'
    elif NO==0:
        return SI,'SI'
    return SI/NO, 'SI'
baseP.groupby(by = ['NOM_PREGUNTA','NOM_PROVINCIA','NOM_OPCION']).sum(numeric_only=True)[['FIN_RESULTADO']].reset_index()
resultado_prop = pd.pivot_table(baseP,'FIN_RESULTADO',aggfunc='sum',index=['NOM_PREGUNTA','NOM_PROVINCIA','NOM_CANTON','COD_PARROQUIA'],columns='NOM_OPCION').reset_index()
resultado_prop['prop'] = resultado_prop.apply(lambda row: proporcion(row['NO'],row['SI']),axis=1)
resultado_prop = pd.concat([resultado_prop,resultado_prop['prop'].apply(lambda x: pd.Series(str(x).strip("()'").split(", '")))],axis=1)
resultado_prop['Normalizacion'] = resultado_prop[0].astype(float)/5
resultado_prop['TOTAL'] = resultado_prop['NO'] + resultado_prop['SI']
resultado_prop['NO_prop'] = resultado_prop['NO']/resultado_prop['TOTAL']
resultado_prop['SI_prop'] = resultado_prop['SI']/resultado_prop['TOTAL']
resultado_prop['NO %'] = resultado_prop['NO_prop'].apply(lambda x: f"{x:.2%}")
resultado_prop['SI %'] = resultado_prop['SI_prop'].apply(lambda x: f"{x:.2%}")
dic_SI = resultado_prop[(resultado_prop['NOM_PREGUNTA']=='D - REFERÉNDUM - PREGUNTA 3') & (resultado_prop[1]=='SI')][['NOM_PROVINCIA','Normalizacion']]
dic_SI = dic_SI.set_index('NOM_PROVINCIA').to_dict()['Normalizacion']
dic_NO = resultado_prop[(resultado_prop['NOM_PREGUNTA']=='D - REFERÉNDUM - PREGUNTA 3') & (resultado_prop[1]=='NO')][['NOM_PROVINCIA','Normalizacion']]
dic_NO = dic_NO.set_index('NOM_PROVINCIA').to_dict()['Normalizacion']
resultado_prop
vertical = pd.DataFrame()
for i in preguntas:
    if i == 'A - REFERÉNDUM - PREGUNTA':
        letra = i[0]
        aux = resultado_prop[resultado_prop['NOM_PREGUNTA']==i][[ 'NOM_PREGUNTA', 'NOM_PROVINCIA',    'NOM_CANTON', 'COD_PARROQUIA',
                  'NO',            'SI',          'TOTAL',       'NO_prop',
             'SI_prop']]
        aux.columns = [ 'NOM_PREGUNTA', 'NOM_PROVINCIA',    'NOM_CANTON', 'COD_PARROQUIA',
                  f'{letra}_NO',            f'{letra}_SI',          f'{letra}_TOTAL',       f'{letra}_NO_prop',
             f'{letra}_SI_prop']
        vertical = pd.concat([vertical,aux])
    else:
        letra = i[0]
        aux = resultado_prop[resultado_prop['NOM_PREGUNTA']==i][['COD_PARROQUIA',
                  'NO',            'SI',          'TOTAL',       'NO_prop',
             'SI_prop']]
        aux.columns = [ 'COD_PARROQUIA',
                  f'{letra}_NO',            f'{letra}_SI',          f'{letra}_TOTAL',       f'{letra}_NO_prop',
             f'{letra}_SI_prop']
        vertical = pd.merge(vertical,aux,on='COD_PARROQUIA')
        
vertical.drop('NOM_PREGUNTA',axis=1)

WARNING *** file size (2252492) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,NOM_PROVINCIA,NOM_CANTON,COD_PARROQUIA,A_NO,A_SI,A_TOTAL,A_NO_prop,A_SI_prop,B_NO,B_SI,...,J_NO,J_SI,J_TOTAL,J_NO_prop,J_SI_prop,K_NO,K_SI,K_TOTAL,K_NO_prop,K_SI_prop
0,AMERICA LATINA EL CARIBE Y AFRICA,ARGENTINA ...,9003,113,684,797,0.141782,0.858218,128,667,...,145,649,794,0.182620,0.817380,177,617,794,0.222922,0.777078
1,AMERICA LATINA EL CARIBE Y AFRICA,BOLIVIA ...,9008,7,21,28,0.250000,0.750000,9,18,...,7,20,27,0.259259,0.740741,7,20,27,0.259259,0.740741
2,AMERICA LATINA EL CARIBE Y AFRICA,BRASIL ...,9009,3,36,39,0.076923,0.923077,5,34,...,5,33,38,0.131579,0.868421,4,34,38,0.105263,0.894737
3,AMERICA LATINA EL CARIBE Y AFRICA,BRASIL ...,9067,68,132,200,0.340000,0.660000,89,110,...,77,123,200,0.385000,0.615000,85,115,200,0.425000,0.575000
4,AMERICA LATINA EL CARIBE Y AFRICA,CHILE ...,9019,215,1556,1771,0.121400,0.878600,296,1467,...,275,1265,1540,0.178571,0.821429,330,1421,1751,0.188464,0.811536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,ZAMORA CHINCHIPE,ZAMORA ...,2155,161,575,736,0.218750,0.781250,213,502,...,200,490,690,0.289855,0.710145,235,443,678,0.346608,0.653392
1291,ZAMORA CHINCHIPE,ZAMORA ...,3260,118,424,542,0.217712,0.782288,175,354,...,175,348,523,0.334608,0.665392,183,340,523,0.349904,0.650096
1292,ZAMORA CHINCHIPE,ZAMORA ...,4010,196,1083,1279,0.153245,0.846755,267,986,...,234,1009,1243,0.188254,0.811746,286,957,1243,0.230088,0.769912
1293,ZAMORA CHINCHIPE,ZAMORA ...,6305,1423,8501,9924,0.143390,0.856610,1872,8029,...,1818,8020,9838,0.184794,0.815206,2034,7794,9828,0.206960,0.793040


In [27]:
resultado_prop

,NOM_PREGUNTA,NOM_PROVINCIA,NOM_CANTON,COD_PARROQUIA,NO,SI,prop,0,1,Normalizacion,TOTAL,NO_prop,SI_prop,NO %,SI %
0,A - REFERÉNDUM - PREGUNTA,AMERICA LATINA EL CARIBE Y AFRICA,ARGENTINA ...,9003,113,684,"(6.053097345132743, SI)",6.053097345132743,SI,1.210619,797,0.141782,0.858218,14.18%,85.82%
1,A - REFERÉNDUM - PREGUNTA,AMERICA LATINA EL CARIBE Y AFRICA,BOLIVIA ...,9008,7,21,"(3.0, SI)",3.0,SI,0.600000,28,0.250000,0.750000,25.00%,75.00%
2,A - REFERÉNDUM - PREGUNTA,AMERICA LATINA EL CARIBE Y AFRICA,BRASIL ...,9009,3,36,"(12.0, SI)",12.0,SI,2.400000,39,0.076923,0.923077,7.69%,92.31%
3,A - REFERÉNDUM - PREGUNTA,AMERICA LATINA EL CARIBE Y AFRICA,BRASIL ...,9067,68,132,"(1.9411764705882353, SI)",1.9411764705882353,SI,0.388235,200,0.340000,0.660000,34.00%,66.00%
4,A - REFERÉNDUM - PREGUNTA,AMERICA LATINA EL CARIBE Y AFRICA,CHILE ...,9019,215,1556,"(7.2372093023255815, SI)",7.2372093023255815,SI,1.447442,1771,0.121400,0.878600,12.14%,87.86%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11660,I - CONSULTA POPULAR - PREGUNTA 4,ZAMORA CHINCHIPE,ZAMORA ...,2155,211,474,"(2.2464454976303316, SI)",2.2464454976303316,SI,0.449289,685,0.308029,0.691971,30.80%,69.20%
11661,I - CONSULTA POPULAR - PREGUNTA 4,ZAMORA CHINCHIPE,ZAMORA ...,3260,173,358,"(2.069364161849711, SI)",2.069364161849711,SI,0.413873,531,0.325800,0.674200,32.58%,67.42%
11662,I - CONSULTA POPULAR - PREGUNTA 4,ZAMORA CHINCHIPE,ZAMORA ...,4010,230,1016,"(4.417391304347826, SI)",4.417391304347826,SI,0.883478,1246,0.184591,0.815409,18.46%,81.54%
11663,I - CONSULTA POPULAR - PREGUNTA 4,ZAMORA CHINCHIPE,ZAMORA ...,6305,1841,8008,"(4.349809885931559, SI)",4.349809885931559,SI,0.869962,9849,0.186923,0.813077,18.69%,81.31%


In [26]:
preguntas

array(['A - REFERÉNDUM - PREGUNTA', 'B - REFERÉNDUM - PREGUNTA 1',
       'C - REFERÉNDUM - PREGUNTA 2', 'D - REFERÉNDUM - PREGUNTA 3',
       'E - REFERÉNDUM - PREGUNTA 4', 'F - CONSULTA POPULAR - PREGUNTA 1',
       'G - CONSULTA POPULAR - PREGUNTA 2',
       'H - CONSULTA POPULAR - PREGUNTA 3',
       'I - CONSULTA POPULAR - PREGUNTA 4'], dtype=object)

In [17]:
ruta = '/Users/luis/Desktop/TRABAJO/Cloud/ELECCIONES_2024/app/resultados_azure/CATALOGO_2v/'
ruta2 = '/Users/luis/Desktop/TRABAJO/Cloud/ELECCIONES_2024/app/resultados_azure/'
CANDIDATOS = pd.read_excel(f'{ruta}CANDIDATOS.xls')
CANTON = pd.read_excel(f'{ruta}CANTON.xls')
CIRCUNSCRIPCION = pd.read_excel(f'{ruta}CIRCUNSCRIPCION.xls')
DIGNIDAD = pd.read_excel(f'{ruta}DIGNIDAD.xls')
JUNTA = pd.read_excel(f'{ruta}JUNTA.xls')
PARROQUIA = pd.read_excel(f'{ruta}PARROQUIA.xls')
PARTIDO = pd.read_excel(f'{ruta}PARTIDO.xls')
PROVINCIA = pd.read_excel(f'{ruta}PROVINCIA.xls')
ZONA = pd.read_excel(f'{ruta}ZONA.xls')
Est_parroquia = pd.read_excel('/Users/luis/Downloads/74_DISTRIBUTIVOS/74_DISTRI_JULIO_2022_PARROQUIA.xls')
Est_parroquia = Est_parroquia[['COD_PARROQUIA','EST_PARROQUIA']]
baseP = pd.read_table(f'{ruta2}PRESIDENTA_E_Y_VICEPRESIDENTA_E_2023.10.23_08.45.01.txt',sep='|',names=['COD_DIGNIDAD','COD_PROVINCIA',
                                                                                       'COD_CIRCUNSCRIPCION','COD_CANTON','COD_PARROQUIA',
                                                                                       'COD_ZONA','NUM_JUNTA','SEX_JUNTA','COD_JUNTA',
                                                                                       'ORDEN','COD_PARTIDO','NUM_SUF_ACTA','BLANCOS','NULOS','FIN_RESULTADO'])
baseP = pd.merge(DIGNIDAD,baseP,on='COD_DIGNIDAD')
baseP = pd.merge(PROVINCIA,baseP,on='COD_PROVINCIA')
baseP = pd.merge(CANTON[['COD_CANTON','NOM_CANTON']],baseP,on='COD_CANTON')
baseP = pd.merge(PARROQUIA[['COD_PARROQUIA','NOM_PARROQUIA']],baseP,on='COD_PARROQUIA')
baseP = pd.merge(Est_parroquia,baseP,on='COD_PARROQUIA')
baseP = pd.merge(PARTIDO,baseP,on='COD_PARTIDO')
resultadosP = baseP.groupby(by = ['NOM_PARTIDO']).sum()[['FIN_RESULTADO']].reset_index()
resultadosP['NOM_CANDIDATO']= ''
resultadosP.loc[resultadosP['NOM_PARTIDO']=='ACCIÓN DEMOCRÁTICA NACIONAL, ADN','NOM_CANDIDATO'] = 'DANIEL NOBOA'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='ACTUEMOS','NOM_CANDIDATO'] = 'OTTO SONNENHOLZNER'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='ALIANZA POR UN PAÍS SIN MIEDO, LISTAS 6-3-1','NOM_CANDIDATO'] = 'JAN TOPIC'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='CLARO QUE SE PUEDE','NOM_CANDIDATO'] = 'YAKU PEREZ'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES','NOM_CANDIDATO'] = 'BOLIVAR ARMIJOS'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='MOVIMIENTO CONSTRUYE','NOM_CANDIDATO'] = 'FERNANDO VILLAVICENCIO'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='MOVIMIENTO POLÍTICO REVOLUCIÓN CIUDADANA','NOM_CANDIDATO'] = 'LUISA GONZALEZ'
resultadosP.loc[resultadosP['NOM_PARTIDO']=='MOVIMIENTO RENOVACIÓN TOTAL, RETO','NOM_CANDIDATO'] = 'XAVIER HERVAS'

resultadosP#.to_excel('Resultados Presidenciales Desagregados 14h30.xlsx')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


/var/folders/0q/l5j608j10jq82grhg4m4pf0h0000gn/T/ipykernel_89405/2437918677.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  resultadosP = baseP.groupby(by = ['NOM_PARTIDO']).sum()[['FIN_RESULTADO']].reset_index()


,NOM_PARTIDO,FIN_RESULTADO,NOM_CANDIDATO
0,"ACCIÓN DEMOCRÁTICA NACIONAL, ADN",5250872,DANIEL NOBOA
1,MOVIMIENTO POLÍTICO REVOLUCIÓN CIUDADANA,4879682,LUISA GONZALEZ


In [23]:
resultados2vuelta = baseP.pivot_table(index='COD_PARROQUIA',columns='NOM_PARTIDO',aggfunc='sum')['FIN_RESULTADO'].reset_index()
resultados2vuelta.columns = ['COD_PARROQUIA','ADN','RC']
resultados2vuelta['TOTAL_2V'] = resultados2vuelta['ADN'] + resultados2vuelta['RC']
resultados2vuelta['ADN_prop'] = resultados2vuelta['ADN']/resultados2vuelta['TOTAL_2V']
resultados2vuelta['RC_prop'] = resultados2vuelta['RC']/resultados2vuelta['TOTAL_2V']
resultados2vuelta
#baseP.groupby(by = ['NOM_PARTIDO']).sum()[['FIN_RESULTADO']].reset_index()

/var/folders/0q/l5j608j10jq82grhg4m4pf0h0000gn/T/ipykernel_89405/1067752846.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  resultados2vuelta = baseP.pivot_table(index='COD_PARROQUIA',columns='NOM_PARTIDO',aggfunc='sum')['FIN_RESULTADO'].reset_index()


,COD_PARROQUIA,ADN,RC,TOTAL_2V,ADN_prop,RC_prop
0,5,300,787,1087,0.275989,0.724011
1,10,3156,7299,10455,0.301865,0.698135
2,15,1026,764,1790,0.573184,0.426816
3,20,1356,3819,5175,0.262029,0.737971
4,25,3113,1251,4364,0.713336,0.286664
...,...,...,...,...,...,...
1291,9073,57,18,75,0.760000,0.240000
1292,9075,10,9,19,0.526316,0.473684
1293,9077,3,0,3,1.000000,0.000000
1294,9078,5,3,8,0.625000,0.375000


In [33]:
resultado_comparacion = pd.merge(vertical,resultados2vuelta,on='COD_PARROQUIA').merge(PARROQUIA,on='COD_PARROQUIA')
resultado_comparacion[['NOM_PREGUNTA', 'NOM_PROVINCIA', 'NOM_CANTON', 'NOM_PARROQUIA','COD_PARROQUIA', 'A_NO',
       'A_SI', 'A_TOTAL', 'A_NO_prop', 'A_SI_prop', 'B_NO', 'B_SI', 'B_TOTAL',
       'B_NO_prop', 'B_SI_prop', 'C_NO', 'C_SI', 'C_TOTAL', 'C_NO_prop',
       'C_SI_prop', 'D_NO', 'D_SI', 'D_TOTAL', 'D_NO_prop', 'D_SI_prop',
       'E_NO', 'E_SI', 'E_TOTAL', 'E_NO_prop', 'E_SI_prop', 'F_NO', 'F_SI',
       'F_TOTAL', 'F_NO_prop', 'F_SI_prop', 'G_NO', 'G_SI', 'G_TOTAL',
       'G_NO_prop', 'G_SI_prop', 'H_NO', 'H_SI', 'H_TOTAL', 'H_NO_prop',
       'H_SI_prop', 'I_NO', 'I_SI', 'I_TOTAL', 'I_NO_prop', 'I_SI_prop',
       'J_NO', 'J_SI', 'J_TOTAL', 'J_NO_prop', 'J_SI_prop', 'K_NO', 'K_SI',
       'K_TOTAL', 'K_NO_prop', 'K_SI_prop', 'ADN', 'RC', 'TOTAL_2V',
       'ADN_prop', 'RC_prop', 'COD_CANTON' ]].to_excel('Comparativa_final_2v_vs_referendum_consulta.xlsx',index=False)